### checking


In [94]:
import duckdb

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df()


description

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


a) Which movies are longer than 3 hours (180 minutes), show the title and its length?


In [95]:
description.column_names[9]

array(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update',
       '_dlt_load_id', '_dlt_id'], dtype=object)

In [96]:
with duckdb.connect("data/sakila.duckdb") as conn:
    film_length = conn.sql("SELECT title, length FROM staging.film WHERE length > 180 ORDER BY length DESC;").df()

film_length

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


b) Which movies have the word "love" in its title? Show the following columns

In [97]:
with duckdb.connect("data/sakila.duckdb") as conn:
    love_in_title = conn.sql("SELECT title, rating, length, description FROM staging.film WHERE regexp_matches(title, '(?i)\\blove\\b')""").df()
# some \ and new " " appeared when I went to the next line - to check with Debbie if normal
love_in_title
#Just love in title
# capital letters so no need for ILIKE



,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and
longest movie length

In [98]:
# from 04_querying_data/sql/aggregate_functions.sql:
# MIN() AS shortest_duration,
# AVG() AS mean_duration,
# MEDIAN() AS median_duration,
# MAX() AS longest_duration
with duckdb.connect("data/sakila.duckdb") as conn:
    film_duration = conn.sql("SELECT MIN(length) as shortest_duration, ROUND(AVG(length)) as mean_duration, MEDIAN(length) as median_duration, MAX(length) as longest_duration FROM staging.film;").df()

film_duration

,shortest_duration,mean_duration,median_duration,longest_duration
0,46,115.0,114.0,185


d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can
keep the movie. The Manager wants to know the 10 most expensive movies to rent per day

In [99]:
with duckdb.connect("data/sakila.duckdb") as conn:
    film_duration = conn.sql("SELECT title, rental_duration, rental_rate, (rental_rate/rental_duration) AS rent_day FROM staging.film ORDER BY rent_day DESC LIMIT 10;").df()

film_duration

,title,rental_duration,rental_rate,rent_day
0,ACE GOLDFINGER,3,4.99,1.663333
1,AMERICAN CIRCUS,3,4.99,1.663333
2,AUTUMN CROW,3,4.99,1.663333
3,BACKLASH UNDEFEATED,3,4.99,1.663333
4,BEAST HUNCHBACK,3,4.99,1.663333
5,BEHAVIOR RUNAWAY,3,4.99,1.663333
6,BILKO ANONYMOUS,3,4.99,1.663333
7,CARIBBEAN LIBERTY,3,4.99,1.663333
8,CASPER DRAGONFLY,3,4.99,1.663333
9,CASUALTIES ENCINO,3,4.99,1.663333


e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have
played in.


In [100]:
with duckdb.connect("data/sakila.duckdb") as conn:
    top_10_actors = conn.sql("SELECT a.actor_id, a.first_name || ' ' || a.last_name AS actor_name, COUNT(fa.film_id) AS movie_count FROM staging.actor a LEFT JOIN staging.film_actor fa ON a.actor_id = fa.actor_id GROUP BY a.actor_id, a.first_name, a.last_name HAVING COUNT(fa.film_id) > 0 ORDER BY movie_count DESC LIMIT 10;").df()
# needed more space in the CONCAT between the " "
# wonder if should keep HAVING COUNT(fa.film_id) > 0? because they won t make the top 10 anyway.
# to change the index?
## not sure about this one
top_10_actors

,actor_id,actor_name,movie_count
0,107.0,GINA DEGENERES,42
1,102.0,WALTER TORN,41
2,198.0,MARY KEITEL,40
3,181.0,MATTHEW CARREY,39
4,23.0,SANDRA KILMER,37
5,81.0,SCARLETT DAMON,36
6,60.0,HENRY BERRY,35
7,13.0,UMA WOOD,35
8,144.0,ANGELA WITHERSPOON,35
9,37.0,VAL BOLGER,35


f) Now it's time for you to choose your own question to explore the sakila database! Write down 3-5
questions you want to answer and then answer them using pandas and duckdb.


In [101]:
#with duckdb.connect('data/sakila.duckdb') as conn:
description
    

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


In [102]:
description.column_names[14]

array(['payment_id', 'customer_id', 'staff_id', 'rental_id', 'amount',
       'payment_date', 'last_update', '_dlt_load_id', '_dlt_id'],
      dtype=object)

Question 1: Which are the top 3 most rented film categories? 

In [103]:
with duckdb.connect('data/sakila.duckdb') as conn:
    top_3_categories = conn.sql("SELECT" \
    " c.name as Category, " \
    "COUNT(r.rental_id) as total_rentals " \
    "FROM staging.category c " \
    "JOIN staging.film_category fc ON c.category_id = fc.category_id " \
    "JOIN staging.film f ON fc.film_id = f.film_id " \
    "JOIN staging.inventory inv ON f.film_id = inv.film_id " \
    "JOIN staging.rental r ON inv.inventory_id = r.inventory_id " \
    "GROUP BY c.name " \
    "ORDER BY total_rentals DESC " \
    "LIMIT 3;"
    ).df()

top_3_categories


,Category,total_rentals
0,Sports,1179
1,Animation,1166
2,Action,1112


Question 2: Who are the churned customers (full name, email, last rental date) to email them a voucher to reactivate them

In [104]:
# Checking the ype for active

with duckdb.connect('data/sakila.duckdb') as conn:
    check_active = conn.sql("DESC staging.customer;").df()

check_active

# Note: active is not Boolean

,column_name,column_type,null,key,default,extra
0,customer_id,BIGINT,NO,None,None,None
1,store_id,BIGINT,NO,None,None,None
2,first_name,VARCHAR,NO,None,None,None
3,last_name,VARCHAR,NO,None,None,None
4,email,VARCHAR,YES,None,None,None
5,address_id,BIGINT,NO,None,None,None
6,active,VARCHAR,NO,None,None,None
7,create_date,TIMESTAMP,NO,None,None,None
8,last_update,TIMESTAMP,NO,None,None,None
9,_dlt_load_id,VARCHAR,NO,None,None,None


In [105]:
# active is a str/VARCHAR 

with duckdb.connect('data/sakila.duckdb') as conn:
    check_active2 = conn.sql("SELECT " \
    "active, " \
    "COUNT(*) as active_count " \
    "FROM staging.customer " \
    "GROUP BY active " \
    "ORDER BY active_count;").df()

check_active2

# It is a str with '0' and '1'

,active,active_count
0,0,15
1,1,584


In [106]:
# Getting the list of the churn customers

with duckdb.connect('data/sakila.duckdb') as conn:
    churn_cust_list = conn.sql("SELECT " \
    "cust.customer_id, " \
    "CONCAT(cust.first_name,' ', cust.last_name) AS customer_name, " \
    "MIN(r.rental_date) as first_rental, " \
    "MAX(r.rental_date) as last_rental, " \
    "cust.email, " \
    "FROM staging.customer cust " \
    "LEFT JOIN staging.rental r ON cust.customer_id =  r.customer_id " \
    "WHERE cust.active = '0' " \
    "GROUP BY ALL " \
    "ORDER BY customer_name;").df()

churn_cust_list

,customer_id,customer_name,first_rental,last_rental,email
0,510,BEN EASTER,2005-05-25 11:13:34,2005-08-22 10:46:44,BEN.EASTER@sakilacustomer.org
1,534,CHRISTIAN JUNG,2005-05-26 21:21:28,2006-02-14 15:16:03,CHRISTIAN.JUNG@sakilacustomer.org
2,169,ERICA MATTHEWS,2005-05-28 04:28:38,2005-08-23 09:33:22,ERICA.MATTHEWS@sakilacustomer.org
3,368,HARRY ARCE,2005-05-25 09:21:29,2005-08-23 00:31:09,HARRY.ARCE@sakilacustomer.org
4,241,HEIDI LARSON,2005-05-28 17:04:43,2005-08-23 13:14:15,HEIDI.LARSON@sakilacustomer.org
5,558,JIMMIE EGGLESTON,2005-06-17 09:19:52,2005-08-21 06:53:53,JIMMIE.EGGLESTON@sakilacustomer.org
6,64,JUDITH COX,2005-05-28 00:39:31,2006-02-14 15:16:03,JUDITH.COX@sakilacustomer.org
7,315,KENNETH GOODEN,2005-05-28 06:20:55,2006-02-14 15:16:03,KENNETH.GOODEN@sakilacustomer.org
8,482,MAURICE CRAWLEY,2005-05-26 15:32:46,2005-08-23 16:42:53,MAURICE.CRAWLEY@sakilacustomer.org
9,406,NATHAN RUNYON,2005-05-30 02:00:28,2005-08-21 17:45:52,NATHAN.RUNYON@sakilacustomer.org


Question 3: Which stores generated the least revenues in 2006?

In [107]:
# How many stores are there? 

with duckdb.connect('data/sakila.duckdb') as conn:
    count_stores = conn.sql("SELECT COUNT(*) AS store_count FROM staging.store;").df()
count_stores

,store_count
0,2


In [108]:
# Checking rental column 

description.column_names[15]

array(['rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'last_update', '_dlt_load_id',
       '_dlt_id'], dtype=object)

In [109]:
# How many business years?

with duckdb.connect('data/sakila.duckdb') as conn:
    rental_years = conn.sql("SELECT DISTINCT YEAR(rental_date) as rental_year FROM staging.rental ORDER BY rental_year;").df()
rental_years

,rental_year
0,2005
1,2006


In [110]:
with duckdb.connect('data/sakila.duckdb') as conn:
    revenue_stores_2006 = conn.sql("SELECT " \
    "s.store_id, " \
    "CONCAT(ad.address, ', ', ci.city, ', ', cou.country) AS full_address, " \
    "SUM(p.amount) AS total_revenue, " \
    "FROM staging.payment p " \
    "JOIN staging.rental r ON p.rental_id = r.rental_id " \
    "JOIN staging.inventory i ON r.inventory_id = i.inventory_id " \
    "JOIN staging.store s ON i.store_id = s.store_id " \
    "JOIN staging.address ad ON s.address_id = ad.address_id " \
    "JOIN staging.city ci ON ad.city_id = ci.city_id " \
    "JOIN staging.country cou ON ci.country_id = cou.country_id " \
    "WHERE EXTRACT(YEAR FROM p.payment_date) = 2006 " \
    "GROUP BY s.store_id, ad.address, ci.city, cou.country  " \
    "ORDER BY total_revenue ASC;").df()

revenue_stores_2006   
                                  

,store_id,full_address,total_revenue
0,1,"47 MySakila Drive, Lethbridge, Canada",243.10
1,2,"28 MySQL Boulevard, Woodridge, Australia",271.08


In [111]:
# Checking the above for 2006 as very low revenues

with duckdb.connect('data/sakila.duckdb') as conn:
    sales_check_2006 = conn.sql("SELECT " \
    "s.store_id, " \
    "EXTRACT(MONTH FROM p.payment_date) AS month, " \
    "COUNT(p.payment_id) AS rentals_transaction, " \
    "ROUND(SUM(p.amount), 2) AS revenue, " \
    "FROM staging.payment p " \
    "JOIN staging.customer cust ON cust.customer_id = p.customer_id " \
    "JOIN staging.store s ON cust.store_id = s.store_id " \
    "WHERE EXTRACT(YEAR FROM p.payment_date) = 2006 " \
    "GROUP BY s.store_id, EXTRACT(MONTH FROM p.payment_date), " \
    "ORDER BY s.store_id, EXTRACT(MONTH FROM p.payment_date);").df() 

sales_check_2006



,store_id,month,rentals_transaction,revenue
0,1,2,98,283.02
1,2,2,84,231.16


In [ ]:
# Checking if transactions numbers match, should be 182 in 2006 

with duckdb.connect('data/sakila.duckdb') as conn:
    check_monthly_transactions = conn.sql("SELECT " \
    "EXTRACT(YEAR FROM payment_date) AS year, " \
    "EXTRACT(MONTH FROM payment_date) AS month, " \
    "COUNT(p.payment_id) AS transactions, " \
    "FROM staging.payment p " \
    "GROUP BY ALL " \
    "ORDER BY year, month;").df()

check_monthly_transactions

,year,month,transactions
0,2005,5,1157
1,2005,6,2312
2,2005,7,6711
3,2005,8,5687
4,2006,2,182
